# BTC price (91 Days) Data Extraction, Load to BigQuery

In [1]:
# import libraries
import requests 
import pandas as pd
import os
import pandas_gbq

## Data Extraction from Coingecko

In [6]:
# parameters set up for Data Extraction
API_KEY = os.environ.get('GECKO_API_KEY')
if not API_KEY:
    raise ValueError("Error: GECKO_API_KEY environment variable not set.")

COIN_ID = "bitcoin"
DAYS_OF_DATA = "91"

url = f"https://api.coingecko.com/api/v3/coins/{COIN_ID}/market_chart"

In [56]:
params = {
    'vs_currency': 'usd',
    'days': DAYS_OF_DATA,
    'interval': 'daily', 
    'x_cg_demo_api_key': API_KEY
}
print("Attempting to call API with our key...")

# ----------------------------------------------------------------

# request data
response = requests.get(url, params=params)

if response.status_code == 200:
    print("Success! Data received.")
else:
    print(f"Error: Failed to get data. Status code: {response.status_code}")
    print(response.text)

Attempting to call API with our key...
Success! Data received.


In [46]:
data = response.json()
data

{'prices': [[1754784000000, 116510.08393213755],
  [1754870400000, 119266.92516880555],
  [1754956800000, 118773.79960860992],
  [1755043200000, 120202.53485503166],
  [1755129600000, 123560.99363577305],
  [1755216000000, 118405.59579823953],
  [1755302400000, 117339.79190213277],
  [1755388800000, 117501.21653394958],
  [1755475200000, 117542.83687778088],
  [1755561600000, 116256.41276740946],
  [1755648000000, 112778.34483555844],
  [1755734400000, 114252.39755195397],
  [1755820800000, 112414.39987336512],
  [1755907200000, 116834.24948202295],
  [1755993600000, 115359.98346714744],
  [1756080000000, 113399.54847314971],
  [1756166400000, 110185.35443900425],
  [1756252800000, 111842.70999260596],
  [1756339200000, 111216.08479629169],
  [1756425600000, 112525.59740669793],
  [1756512000000, 108480.30666639366],
  [1756598400000, 108781.95727925687],
  [1756684800000, 108253.36092385623],
  [1756771200000, 109162.68557992298],
  [1756857600000, 111190.18209845416],
  [175694400000

In [47]:
# Data view in Dataframe
pd.DataFrame(data['prices']).head()

,0,1
0,1754784000000,116510.083932
1,1754870400000,119266.925169
2,1754956800000,118773.799609
3,1755043200000,120202.534855
4,1755129600000,123560.993636


In [48]:
# selecting just price and date data
price_data = data['prices']

# changing column name
df = pd.DataFrame(price_data, columns=['timestamp', 'price'])
df.head()

,timestamp,price
0,1754784000000,116510.083932
1,1754870400000,119266.925169
2,1754956800000,118773.799609
3,1755043200000,120202.534855
4,1755129600000,123560.993636


In [49]:
# converting date, from milisecond to day_time format
df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
final_df = df[['date', 'price']]

final_df.head()

,date,price
0,2025-08-10,116510.083932
1,2025-08-11,119266.925169
2,2025-08-12,118773.799609
3,2025-08-13,120202.534855
4,2025-08-14,123560.993636


## Loading data into Warehouse (BigQuery)

In [51]:
# setting up parameters to load data to GBQ
GCP_PROJECT_ID = "silken-apex-477018-d4"
destination_table = "dbt_bootcamp.raw_coingecko_bitcoin"
    
print(f"Attempting to load data into BigQuery table: {destination_table}")

# Loading Data
pandas_gbq.to_gbq(
    final_df,
    destination_table=destination_table,
    project_id=GCP_PROJECT_ID,
    if_exists='replace'
)

print("✅ Success! Data loaded into BigQuery.")

Attempting to load data into BigQuery table: dbt_bootcamp.raw_coingecko_bitcoin


100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]

✅ Success! Data loaded into BigQuery.
